<img src="images/splash3.png">
Grabs price and volume for the top 100 cryptos for the past 3 months

## Setup

In [1]:
import sqlite3 as lite
import sys
import coinmarketcap
import requests
import numpy as np
import pandas as pd
import datetime
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option("display.max_rows", 200)

## Fetch data
Uses cryptocompare API to get historical data for target coin the last 3 months, coinmarketcap api to get top-100 coin names. Timestamps are in unix (GMT).

In [2]:
def fetch_top100():
    url = 'https://api.coinmarketcap.com/v2/ticker/'
    page = requests.get(url)
    data = page.json()['data']
    coinlist = pd.DataFrame(data)
    return coinlist

In [3]:
def fetch_coindata(symbol, comparison_symbol, limit, aggregate):    
    url = 'https://min-api.cryptocompare.com/data/histohour?fsym={}&tsym={}&limit={}&aggregate={}'\
            .format(symbol.upper(), comparison_symbol.upper(), limit, aggregate)       
    page = requests.get(url)
    data = page.json()['Data']
    df = pd.DataFrame(data)
    df['mid'] = df[["high", "low"]].mean(axis=1)
    df['timestamp'] = [datetime.datetime.fromtimestamp(d).strftime('%d-%m-%Y %H:%M:%S') for d in df.time]
    #df = df.set_index('timestamp')        
    return df

## Store values to database

In [4]:
def write_to_db():
    #Connect to DB
    con = None
    con = lite.connect('crypto.db')

    #Dump to SQL
    with con:
        market_data.to_sql(name=top_coins[symbol], con=con, if_exists='replace', index=True)

    """"
    #Read SQL    
    with con:        
        cur = con.cursor()    
        cur.execute("SELECT * FROM btc")

        rows = cur.fetchall()

        for row in rows:
            print(row)  
    """

In [8]:
top_coins = fetch_top100().loc['symbol']

for symbol in range(1, 100):
    try:
        market_data = fetch_coindata(top_coins[symbol],'BTC','2000','12')
        market_data = market_data.drop(market_data.index[len(market_data)-1]) # Only keep the latest timestamped column
        write_to_db()
    except:
        print("Error noticed, skipping coin ", top_coins[symbol])
        
print("Success!")

Error noticed, skipping coin  MIOTA
Error noticed, skipping coin  CENNZ
Success!
